<a href="https://colab.research.google.com/github/PacktPublishing/Machine-Learning-for-Time-Series-with-Python/blob/master/chapter11/Trading_with_DQN.ipynb" target="_parent\"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
# based on https://github.com/tensortrade-org/tensortrade/blob/master/examples/train_and_evaluate.ipynb

In [57]:
#pip install git+https://github.com/tensortrade-org/tensortrade.git

In [58]:
# all imports:
import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, ETH
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.agents import DQNAgent


%matplotlib inline

In [59]:
cdd = CryptoDataDownload()

data = cdd.fetch("Bitstamp", "USD", "BTC", "1h")

In [60]:
data.head()

,date,unix,open,high,low,close,volume
0,2018-05-15 06:00:00,1526364000,8733.86,8796.68,8707.28,8740.99,559.93
1,2018-05-15 07:00:00,1526367600,8740.99,8766.00,8721.11,8739.00,273.58
2,2018-05-15 08:00:00,1526371200,8739.00,8750.27,8660.53,8728.49,917.79
3,2018-05-15 09:00:00,1526374800,8728.49,8754.40,8701.35,8708.32,182.62
4,2018-05-15 10:00:00,1526378400,8708.32,8865.00,8695.11,8795.90,1260.69


In [61]:
# we'll create a couple of indicators:
def rsi(price: Stream[float], period: float) -> Stream[float]:
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Stream[float], fast: float, slow: float, signal: float) -> Stream[float]:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal## Create features with the feed module

In [62]:
# choosing the closing price:
features = []
for c in data.columns[1:]:
    s = Stream.source(list(data[c]), dtype="float").rename(data[c].name)
    features += [s]

cp = Stream.select(features, lambda s: s.name == "close")

In [63]:
# adding the three features (trend indicator, RSI, MACD):
features = [
    cp.log().diff().rename("lr"),
    rsi(cp, period=20).rename("rsi"),
    macd(cp, fast=10, slow=50, signal=5).rename("macd")
]

feed = DataFeed(features)
feed.compile()

In [64]:
for i in range(5):
    print(feed.next())

{'lr': nan, 'rsi': nan, 'macd': 0.0}
{'lr': -0.00022768891842694927, 'rsi': 0.0, 'macd': -0.1891859774210995}
{'lr': -0.0012033785355889393, 'rsi': 0.0, 'macd': -1.2726616061000744}
{'lr': -0.0023134975946028646, 'rsi': 0.0, 'macd': -3.6577343503541435}
{'lr': 0.01000681330867259, 'rsi': 74.26253567956897, 'macd': 3.7087743627464844}


In [65]:
# setting up broker and the portfolio:
bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(data["close"]), dtype="float").rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(bitstamp, 10000 * USD),
    Wallet(bitstamp, 10 * BTC)
])

In [66]:
# renderer:
renderer_feed = DataFeed([
    Stream.source(list(data["date"])).rename("date"),
    Stream.source(list(data["open"]), dtype="float").rename("open"),
    Stream.source(list(data["high"]), dtype="float").rename("high"),
    Stream.source(list(data["low"]), dtype="float").rename("low"),
    Stream.source(list(data["close"]), dtype="float").rename("close"), 
    Stream.source(list(data["volume"]), dtype="float").rename("volume") 
])

In [67]:
# the trading environment:
env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    renderer_feed=renderer_feed,
    renderer=default.renderers.PlotlyTradingChart(),
    window_size=20
)

In [68]:
env.observer.feed.next()

{'internal': {'bitstamp:/USD-BTC': 8740.99,
  'bitstamp:/USD:/free': 10000.0,
  'bitstamp:/USD:/locked': 0.0,
  'bitstamp:/USD:/total': 10000.0,
  'bitstamp:/BTC:/free': 10.0,
  'bitstamp:/BTC:/locked': 0.0,
  'bitstamp:/BTC:/total': 10.0,
  'bitstamp:/BTC:/worth': 87409.9,
  'net_worth': 97409.9},
 'external': {'lr': nan, 'rsi': nan, 'macd': 0.0},
 'renderer': {'date': Timestamp('2018-05-15 06:00:00'),
  'open': 8733.86,
  'high': 8796.68,
  'low': 8707.28,
  'close': 8740.99,
  'volume': 559.93}}

In [69]:

import numpy as np
import tensorflow as tf
from collections import deque
import random

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = env.observation_space.shape  # pas .shape[0]
        self.action_size = action_size
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.batch_size = 32
        self.memory = deque(maxlen=2000)
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.state_size),  # ex: (20, 3)
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
                      loss='mse')
        return model


    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state[np.newaxis], verbose=0)
        return np.argmax(q_values[0])

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        minibatch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state[np.newaxis], verbose=0)[0]
            if done:
                target[action] = reward
            else:
                t = self.target_model.predict(next_state[np.newaxis], verbose=0)[0]
                target[action] = reward + self.gamma * np.amax(t)
            self.model.fit(state[np.newaxis], target[np.newaxis], epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def train(self, env, episodes=100):
        for e in range(episodes):
            state = env.reset()
            done = False
            total_reward = 0
            while not done:
                action = self.act(state)
                next_state, reward, terminated, truncated, _ = env.step(action)
                done = terminated or truncated
                self.remember(state, action, reward, next_state, done)
                state = next_state
                total_reward += reward
                self.replay()
            self.update_target_model()
            print(f"Episode {e+1}/{episodes}, Total reward: {total_reward}, Epsilon: {self.epsilon:.4f}")


obs_space = env.observation_space
state_size = obs_space.shape if hasattr(obs_space, "shape") and obs_space.shape is not None else (1,)
action_size = env.action_space.n

agent = DQNAgent(state_size=state_size, action_size=action_size)


W0000 00:00:1748602598.666451   96844 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [73]:
%matplotlib inline

performance = pd.DataFrame.from_dict(env.action_scheme.portfolio.performance, orient='index')
performance.plot()

<Axes: >

In [72]:
performance["net_worth"].plot()

<Axes: >